# Similarity Score

In [19]:
from zalign import ZScore
from rdkit import Chem


fp_sdf_q = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/CAT-13g.sdf"
fp_sdf_t = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/reference.sdf"

mol_q = Chem.SDMolSupplier(fp_sdf_q, removeHs=False)[0]
mol_t = Chem.SDMolSupplier(fp_sdf_t, removeHs=False)[0]

matrix = ZScore(mol_q, mol_t).score()

In [21]:
# 用plotly绘制相似性得分矩阵的热力图，可以交互式查看横纵坐标
import plotly.graph_objs as go
import numpy as np

# 构造x轴和y轴标签（原子索引从1开始）
x_labels = [str(i+1) for i in range(matrix.shape[1])]
y_labels = [str(i+1) for i in range(matrix.shape[0])]

heatmap = go.Heatmap(
    z=matrix,
    x=x_labels,
    y=y_labels,
    colorscale='Hot',
    colorbar=dict(title='Score'),
    hoverongaps=False,
    hovertemplate='Query Atom: %{y}<br>Template Atom: %{x}<br>Score: %{z:.1f}<extra></extra>'
)

layout = go.Layout(
    title='Similarity Score Matrix',
    xaxis=dict(title='Template Molecule Atom Index'),
    yaxis=dict(title='Query Molecule Atom Index'),
    width=900,
    height=900
)

fig = go.Figure(data=[heatmap], layout=layout)
fig.show()

## 准备输入JSON

In [ ]:
import json
fp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/tmp/docking_lccdp-S1_2988814_20250822_120329_866547_0bp6cb90/ud2_engine_inputs.json"
fp_align = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/ud2_align.json"

with open(fp, "r") as f:
    data = json.load(f)

k = list(set(data.keys()) - {"score", "receptor"})[0]

# 得到所有原子的坐标
coords_t = mol_t.GetConformer().GetPositions().tolist()

In [ ]:
natom_t = mol_t.GetNumAtoms()
natom_q = mol_q.GetNumAtoms()

for i in range(natom_q):

    biases = []
    for j in range(natom_t): # each template atom
        biases.append([*coords_t[j], float(matrix[i][j]), 0.])
    
    # add biases to this
    data[k]["atoms"][i].append(biases)

with open(fp_align, "w") as f:
    json.dump(data, f)



TypeError: dump() missing 1 required positional argument: 'fp'

# Utils

## Transfer .MOL file to .SDF File

In [22]:
def transfer_mol_to_sdf(mol_file_path):
    fp_sdf = mol_file_path.replace(".mol", ".sdf")
    mol = Chem.MolFromMolFile(mol_file_path)
    Chem.MolToMolFile(mol, fp_sdf)
    return fp_sdf

transfer_mol_to_sdf("/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/test_all_rings.mol")

'/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/test_all_rings.sdf'

# Attractive Energy

# Repulsive Energy